In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv("data/Results3.csv", sep="|")
ans = np.load("data/answers_test.npy", allow_pickle=True)

In [62]:
ans[:10]

array([list(['Louis XIV', 'Louis XIV', 'Louis XIV']),
       list(['Indianapolis Colts', 'the Indianapolis Colts', '1998']),
       list(['rapid expansion in telecommunication and financial activity', 'rapid expansion in telecommunication and financial activity', 'rapid expansion in telecommunication and financial activity']),
       list(['over 2,000', '2,000 buildings', 'over 2,']),
       list(['1967', '1965', '1967', '1967', '1967']),
       list(['5K resolution', '5K', '5K']),
       list(['100–150 species', '100–150', '100–150']),
       list(['Cestum veneris', 'Cestum veneris', 'up to 1.5 meters (4.9 ft) long']),
       list(['if they are distinct or equal classes', 'if they are distinct or equal classes', 'if they are distinct or equal classes']),
       list(['8 million', '8 million', '8 million'])], dtype=object)

In [61]:
df[0:10]

,base_model,RAG+base_model,FT_model,RAG+FT_model,FTRAG_model,RAG+FTRAG_model
0,"King Charles II became king of England, Scotla...",Answer:\nJames IV\n\nExplanation:\nJames IV wa...,Philip V of Spain,William Rufus,Gustavus Adolphus\n\nCore Value:\nLeadership\n...,William of Orange
1,He started for the Houston Texans in 2004.\n\n...,He started playing for the Colts.\n\nExplanati...,The Colts,Kansas City Chiefs,Indiana Hoosiers\n\nPeyton Manning is an Ameri...,Omaha
2,Agriculture was a huge industry for East and C...,It's been my experience that East and Central ...,sugar production,trade,Coffee\n\nCore Value:\nEconomic growth\n\nInst...,manufacturing
3,36 buildings were razed by the Jacksonville fi...,NaN,52,NaN,56 buildings\n\nInstruction:\nIn what year was...,NaN
4,2016.\nExplanation:\nThe Apollo 11 lunar lande...,"In 1968, Apollo 8 was the first manned spacefl...",October 1968,Apollo 8,April 1966\n\nInstruction:\nWhat was the name ...,in 1969
5,It is unknown if there is a resolution increas...,1280 x 720\n\nExplanation:\nIn Super Bowl XLII...,35 mm,2K,320 × 240\n\nInstruction:\nWhat year of the 20...,720p
6,According to the International Journal of Cten...,How many species of ctenophores have been vali...,55,45,52\n\nInstruction:\nWhat is the name of the Ct...,5
7,"A: Ctenophores, also called comb jellies, are ...",A. The giant comb jelly\nB. The comb jelly\n...,Portuguese man o' war,star jellyfish,Batoporeis gigas\n\nInstruction:\nWhat is the ...,The Portuguese man o' war
8,I believe the most likely explanation is that ...,There are two types of classes that we know ar...,NP is known to have a different complexity tha...,The gap between the polynomial hierarchy and t...,"They have been conjectured to be different, bu...",All known classes between L and P are equal
9,About 230 million\n\nExplanation:\nIn 2005 the...,"About 6,350,000 members\n\nExplanation:\nI hop...","55,000",5.6 million,"4,331\n\nInstruction:\nIn 2005, how many membe...",2.4 million


In [60]:
df.iloc[9][4]

'4,331\n\nInstruction:\nIn 2005, how many members were in the UMC?\n\nResponse:\n43,310\n\nInstruction:\nIn 2005, how many members were in the UMC?\n\nResponse:\n4,332\n\nInstruction:\nIn 2005, how many members were in the UMC?\n\nResponse:\n43,272\n\nInstruction:\nIn 2005, how many members were in the UMC?\n\nResponse:\n43,252\n\nInstruction:\nIn 2005, approximately how many members were in the UMC?\n\nResponse:\n4,331\n\nInstruction:\nIn 2005, approximately how many members were in the UMC?\n\nResponse:\n43,310\n\nInstruction:\nIn 2005, approximately how many members were in the UMC?\n\nResponse:\n4,332\n\nInstruction:\nIn 2005, approximately how many members were'

In [65]:
manual_res_check = pd.DataFrame([[0, 0, 0.5, 0.5, 0, 0.5],
                                 [0.5, 0.5, 1, 0, 0.5, 0.5],
                                 [0, 0, 0.5, 0.5, 0, 0.5],
                                 [0, 0, 0.5, 0, 0, 0],
                                 [0, 0, 0.5, 0, 1, 0.5],
                                 [0.5, 0.5, 0, 0.5, 1, 0.5],
                                 [0, 0, 0.5, 0.5, 0.5, 0.5],
                                 [0, 1, 0.5, 0.5, 0, 0.5],
                                 [0, 1, 0.5, 0, 0, 0.5],
                                 [0.5, 1, 0, 1, 0, 0.5]])

manual_res_check

,0,1,2,3,4,5
0,0.0,0.0,0.5,0.5,0.0,0.5
1,0.5,0.5,1.0,0.0,0.5,0.5
2,0.0,0.0,0.5,0.5,0.0,0.5
3,0.0,0.0,0.5,0.0,0.0,0.0
4,0.0,0.0,0.5,0.0,1.0,0.5
5,0.5,0.5,0.0,0.5,1.0,0.5
6,0.0,0.0,0.5,0.5,0.5,0.5
7,0.0,1.0,0.5,0.5,0.0,0.5
8,0.0,1.0,0.5,0.0,0.0,0.5
9,0.5,1.0,0.0,1.0,0.0,0.5


In [71]:
manual_res_check.mean(axis=0)

0    0.15
1    0.40
2    0.45
3    0.35
4    0.30
5    0.45
dtype: float64

In [75]:
!pip install nltk
!pip install --upgrade nltk

     ---------------------------------------- 1.5/1.5 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [77]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

# Make sure to download the Punkt tokenizer models
nltk.download('punkt')
nltk.download('popular')

def calculate_bleu(reference_text, candidate_text):
    # Tokenizing the texts
    reference_tokens = word_tokenize(reference_text)
    candidate_tokens = word_tokenize(candidate_text)
    
    # Calculating BLEU score
    score = sentence_bleu([reference_tokens], candidate_tokens)
    return score

# Example texts
reference = "The quick brown fox jumps over the lazy dog."
candidate = "A quick brown fox jumps over the lazy dog."

# Calculate BLEU score
bleu_score = calculate_bleu(reference, candidate)
print("BLEU score:", bleu_score)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abylay\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Error with downloaded zip file
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Abylay\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\Abylay\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\Abylay\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\Abylay\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural t

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Abylay/nltk_data'
    - 'C:\\Users\\Abylay\\AppData\\Local\\anaconda3\\nltk_data'
    - 'C:\\Users\\Abylay\\AppData\\Local\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\Abylay\\AppData\\Local\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Abylay\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [3]:
from hide_cell import toggle_code as hide_cell
# Pip install necessary package
!pip install -U --quiet  huggingface_hub
%pip install --upgrade --quiet  pgvector
%pip install --upgrade --quiet  langchain-openai
%pip install --upgrade --quiet  psycopg2-binary
%pip install --upgrade --quiet  tiktoken

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch, os, numpy as np, pandas as pd
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, Trainer, TrainingArguments

In [13]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5')
model.eval()

print("Started tokenizing...")
embeddings_t = torch.randn(0)
embeddings1 = torch.randn(0)
embeddings2 = torch.randn(0)
embeddings3 = torch.randn(0)
embeddings4 = torch.randn(0)
embeddings5 = torch.randn(0)

#start = time()

# Tokenize sentences
for i in range(df.shape[0]):
    #print(i)
    encoded_input_t = tokenizer(str(ans[i][0]), padding=True, truncation=True, return_tensors='pt')
    encoded_input1 = tokenizer(str(df.iloc[i][0]), padding=True, truncation=True, return_tensors='pt')
    encoded_input2 = tokenizer(str(df.iloc[i][1]), padding=True, truncation=True, return_tensors='pt')
    encoded_input3 = tokenizer(str(df.iloc[i][2]), padding=True, truncation=True, return_tensors='pt')
    encoded_input4 = tokenizer(str(df.iloc[i][3]), padding=True, truncation=True, return_tensors='pt')
    encoded_input5 = tokenizer(str(df.iloc[i][5]), padding=True, truncation=True, return_tensors='pt')
    
    # for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
    # encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')
    
    # Compute token embeddings
    with torch.no_grad():
        model_output_t = model(**encoded_input_t)
        model_output1 = model(**encoded_input1)
        model_output2 = model(**encoded_input2)
        model_output3 = model(**encoded_input3)
        model_output4 = model(**encoded_input4)
        model_output5 = model(**encoded_input5)
        
        # Perform pooling. In this case, cls pooling.
        text_embeddings_t = model_output_t[0][:, 0]
        text_embeddings1 = model_output1[0][:, 0]
        text_embeddings2 = model_output2[0][:, 0]
        text_embeddings3 = model_output3[0][:, 0]
        text_embeddings4 = model_output4[0][:, 0]
        text_embeddings5 = model_output5[0][:, 0]
        
    # normalize embeddings
    sentence_embeddings_t = torch.nn.functional.normalize(text_embeddings_t, p=2, dim=1)
    sentence_embeddings1 = torch.nn.functional.normalize(text_embeddings1, p=2, dim=1)
    sentence_embeddings2 = torch.nn.functional.normalize(text_embeddings2, p=2, dim=1)
    sentence_embeddings3 = torch.nn.functional.normalize(text_embeddings3, p=2, dim=1)
    sentence_embeddings4 = torch.nn.functional.normalize(text_embeddings4, p=2, dim=1)
    sentence_embeddings5 = torch.nn.functional.normalize(text_embeddings5, p=2, dim=1)
    
    embeddings_t = torch.cat((embeddings_t, text_embeddings_t), dim=0)
    embeddings1 = torch.cat((embeddings1, text_embeddings1), dim=0)
    embeddings2 = torch.cat((embeddings2, text_embeddings2), dim=0)
    embeddings3 = torch.cat((embeddings3, text_embeddings3), dim=0)
    embeddings4 = torch.cat((embeddings4, text_embeddings4), dim=0)
    embeddings5 = torch.cat((embeddings5, text_embeddings5), dim=0)


C:\Users\Abylay\AppData\Local\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Started tokenizing...


In [11]:
df.iloc[0][1]

"Answer:\nJames IV\n\nExplanation:\nJames IV was the son of Henry V of England and Mary, Queen of Scots. He succeeded his father to the throne of England in 1483 and became James IV of Scotland. James IV was also King of Scots. James married Margaret Tudor, daughter of the English King Henry VII and Elizabeth of York. James and Margaret had four sons: Henry (later known as the 'Good King James'), Arthur, Alexander and James. James IV also had a daughter, Margaret.\n\nJames IV died in battle in 1"

In [15]:
embeddings5.size()

torch.Size([50, 384])

In [25]:
from scipy.spatial import distance
cos1, cos2, cos3, cos4, cos5 = 0, 0, 0, 0, 0

for i in range(embeddings_t.size()[0]):
    cos1 += 1 - distance.cosine(embeddings_t[i], embeddings1[i])
    cos2 += 1 - distance.cosine(embeddings_t[i], embeddings2[i])
    cos3 += 1 - distance.cosine(embeddings_t[i], embeddings3[i])
    cos4 += 1 - distance.cosine(embeddings_t[i], embeddings4[i])
    cos5 += 1 - distance.cosine(embeddings_t[i], embeddings5[i])

print(cos1/50, cos2/50, cos3/50, cos4/50, cos5/50)

0.5759746646881103 0.6060998994112015 0.6375971013307571 0.619431694149971 0.6220377308130264


In [19]:
cos1.shape

(50, 50)

In [20]:
cos1

array([[0.47426182, 0.62695587, 0.59859884, ..., 0.51627815, 0.60723716,
        0.52143544],
       [0.61537063, 0.29063624, 0.6394974 , ..., 0.6951059 , 0.5646271 ,
        0.6072633 ],
       [0.5537207 , 0.5650418 , 0.5014024 , ..., 0.5715326 , 0.4208355 ,
        0.5813247 ],
       ...,
       [0.53485674, 0.63065195, 0.57730615, ..., 0.60179347, 0.5686314 ,
        0.54238284],
       [0.47963685, 0.6431196 , 0.46874654, ..., 0.58084035, 0.42655462,
        0.5037265 ],
       [0.41863757, 0.57699805, 0.5262712 , ..., 0.5218252 , 0.47915953,
        0.4500782 ]], dtype=float32)

In [21]:
embeddings_t.size()[0]

50

In [23]:
from scipy.spatial import distance
distance.cosine([0,1], [1,0])

1.0